## Pyspark Handling Missing Values (Tutorial 3)
* Dropping Columns
* Dropping Rows
* Various Parameter in Dropping functionalities
* Handling missing values by Mean, median, and mode

(Starts at ~31:40 in the video)

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [3]:
df_pyspark = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [4]:
df_pyspark.show()

+------+----+----------+-------+
|  Name| Age|Experience| Salary|
+------+----+----------+-------+
|Graeme|  26|        10|1000000|
|  Rach|  25|         2|1000001|
|  Noah|  28|         5|  50000|
|Taylor|  32|         7|  60000|
|  John|  28|      NULL|  60000|
|  Hunt|NULL|         2|  30000|
|  Rice|NULL|      NULL|  40000|
+------+----+----------+-------+



### Dropping columns

In [5]:
no_name_df = df_pyspark.drop('Name').show()

+----+----------+-------+
| Age|Experience| Salary|
+----+----------+-------+
|  26|        10|1000000|
|  25|         2|1000001|
|  28|         5|  50000|
|  32|         7|  60000|
|  28|      NULL|  60000|
|NULL|         2|  30000|
|NULL|      NULL|  40000|
+----+----------+-------+



In [6]:
# if you do not include any arguments in drop, it will drop all rows that contain any nulls
all_drop_df = df_pyspark.na.drop().show()

+------+---+----------+-------+
|  Name|Age|Experience| Salary|
+------+---+----------+-------+
|Graeme| 26|        10|1000000|
|  Rach| 25|         2|1000001|
|  Noah| 28|         5|  50000|
|Taylor| 32|         7|  60000|
+------+---+----------+-------+



In [7]:
### how = 'all' will only drop when all rows have a null value
### how = 'any' will drop when any values in a row have nulls
df_pyspark.na.drop(how='all').show()
df_pyspark.na.drop(how='any').show()


+------+----+----------+-------+
|  Name| Age|Experience| Salary|
+------+----+----------+-------+
|Graeme|  26|        10|1000000|
|  Rach|  25|         2|1000001|
|  Noah|  28|         5|  50000|
|Taylor|  32|         7|  60000|
|  John|  28|      NULL|  60000|
|  Hunt|NULL|         2|  30000|
|  Rice|NULL|      NULL|  40000|
+------+----+----------+-------+

+------+---+----------+-------+
|  Name|Age|Experience| Salary|
+------+---+----------+-------+
|Graeme| 26|        10|1000000|
|  Rach| 25|         2|1000001|
|  Noah| 28|         5|  50000|
|Taylor| 32|         7|  60000|
+------+---+----------+-------+



In [8]:
## thresh=3 will make it so that only drop rows where there are at least 3 non null values in the row
df_pyspark.na.drop(how='any',thresh=3).show()

+------+----+----------+-------+
|  Name| Age|Experience| Salary|
+------+----+----------+-------+
|Graeme|  26|        10|1000000|
|  Rach|  25|         2|1000001|
|  Noah|  28|         5|  50000|
|Taylor|  32|         7|  60000|
|  John|  28|      NULL|  60000|
|  Hunt|NULL|         2|  30000|
+------+----+----------+-------+



In [12]:
### Using subset
### This example will drop any row that has nulls in the experience column
df_pyspark.na.drop(how='any', subset=['Experience']).show()
type(df_pyspark.columns)

+------+----+----------+-------+
|  Name| Age|Experience| Salary|
+------+----+----------+-------+
|Graeme|  26|        10|1000000|
|  Rach|  25|         2|1000001|
|  Noah|  28|         5|  50000|
|Taylor|  32|         7|  60000|
|  Hunt|NULL|         2|  30000|
+------+----+----------+-------+



list

### Current general idea of how I will be filtering out the libraries for Ning

In [17]:
def has_age_column(spark_df):
    output = ""
    if 'Name' in spark_df.columns:
        output += "Has a name column"
        spark_df.select(['Name']).show() 
        # throw in a where statement to remove the certain types of values we are trying to filter by
    else:
       output +=  "Does not have name column"

    print(output)
    


In [16]:
has_age_column(df_pyspark)

+------+
|  Name|
+------+
|Graeme|
|  Rach|
|  Noah|
|Taylor|
|  John|
|  Hunt|
|  Rice|
+------+

has a name column


In [25]:
### Filling the Missing Value
### this value you are filling with must be the same datatype as the column it is being put into.
### So here all int columns with null values will have it replace with 0
new_df = df_pyspark.na.fill(-1)
new_df.show()

+------+---+----------+-------+
|  Name|Age|Experience| Salary|
+------+---+----------+-------+
|Graeme| 26|        10|1000000|
|  Rach| 25|         2|1000001|
|  Noah| 28|         5|  50000|
|Taylor| 32|         7|  60000|
|  John| 28|        -1|  60000|
|  Hunt| -1|         2|  30000|
|  Rice| -1|        -1|  40000|
+------+---+----------+-------+



In [26]:
# can also fill with a list
df_pyspark.na.fill(-1, ['Age', 'Experience']).show()


+------+---+----------+-------+
|  Name|Age|Experience| Salary|
+------+---+----------+-------+
|Graeme| 26|        10|1000000|
|  Rach| 25|         2|1000001|
|  Noah| 28|         5|  50000|
|Taylor| 32|         7|  60000|
|  John| 28|        -1|  60000|
|  Hunt| -1|         2|  30000|
|  Rice| -1|        -1|  40000|
+------+---+----------+-------+



### replacing a null value with the mean or median of the column

In [37]:
#before
df_pyspark.show()

+------+----+----------+-------+
|  Name| Age|Experience| Salary|
+------+----+----------+-------+
|Graeme|  26|        10|1000000|
|  Rach|  25|         2|1000001|
|  Noah|  28|         5|  50000|
|Taylor|  32|         7|  60000|
|  John|  28|      NULL|  60000|
|  Hunt|NULL|         2|  30000|
|  Rice|NULL|      NULL|  40000|
+------+----+----------+-------+



In [34]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['Age','Experience','Salary'],
    outputCols = ["{}_imputed".format(c) for c in ['Age','Experience', 'Salary']]
).setStrategy("mean") # this could also be median or mode.

In [35]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+----------+-------+-----------+------------------+--------------+
|  Name| Age|Experience| Salary|Age_imputed|Experience_imputed|Salary_imputed|
+------+----+----------+-------+-----------+------------------+--------------+
|Graeme|  26|        10|1000000|         26|                10|       1000000|
|  Rach|  25|         2|1000001|         25|                 2|       1000001|
|  Noah|  28|         5|  50000|         28|                 5|         50000|
|Taylor|  32|         7|  60000|         32|                 7|         60000|
|  John|  28|      NULL|  60000|         28|                 5|         60000|
|  Hunt|NULL|         2|  30000|         27|                 2|         30000|
|  Rice|NULL|      NULL|  40000|         27|                 5|         40000|
+------+----+----------+-------+-----------+------------------+--------------+

